<a href="https://colab.research.google.com/github/Oozankilic/university-projects/blob/main/PrepareData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
activity = pd.read_csv('/content/drive/MyDrive/senior_project/activity.csv', dtype=object)
sleep = pd.read_csv('/content/drive/MyDrive/senior_project/fitbitsleep.csv', dtype=object)

In [6]:
# column names of dataDate are different in sleep and activity dataframe. It is fixed
sleep.rename(columns = {'dataDate':'datadate'}, inplace = True)

# spoiled activity data is deleted
newActivity = activity.drop_duplicates(subset=['egoid','datadate'], keep='first')

# dataframes are inner joined, every tracked day has one activity data and at least one sleep data
merged_df = pd.merge(newActivity, sleep, on=['egoid','datadate'], validate='one_to_many')


In [28]:
# Getting survey data according to the needed waves and preprocessing verbal values

survey = pd.read_csv('/content/drive/MyDrive/senior_project/survey_data.csv', dtype=object)

scores = {
    'Strongly Disagree': 1,
    'Disagree': 2,
    'Neither Agree nor Disagree': 3,
    'Agree': 4,
    'Strongly Agree': 5,
    
    'no': 0,
    'yes': 1,

    'No': 0,
    'Yes': 1,

    'Male': 1,
    'Female': 2,
    
    'Almost never': 1,
    'Sometimes': 2,
    'Often': 3,
    'Almost always': 4,

    'Not at all': 0,
    'Less than 1-2 times a month': 1,
    '1-2 times a month': 2,
    '1-2 times a week': 3,
    'Three times a week or more': 4,

    'Public school': 1, 
    'Public charter school': 2,
    'Public magnet school': 3,
    'Private religious/parochial': 4,
    'Private independent college-prep school': 5,
    'Home school': 6,
    'Other- Please specify': 7,

    'Same sex- male': 1,
    'Same sex- female': 2,
    'A mixture of boys and girls': 3,
    'Not applicable': 4,

    'A or A+': 1,
    'A-': 2,
    'B+': 3,
    'b': 4,
    'B': 4,
    'B-': 5,

    '1-2': 1,
    '3-4': 2,
    '5 or more': 3,
    'Not offered at my high school': 4,
    
    "Bachelor's degree (B.A., B.S., etc.)": 1,
    "Master's Degree (M.A., M.S., MBA, etc.)": 2,
    "Ph.D or Ed.D.": 3,
    'M.D., D.O., D.D.S., or D.V.M.': 4,
    "J.D. (Law)": 3,

    'Living together': 1,
    'Divorced or living apart': 2,

    'only child': 1,
    'first born': 2,
    'second born': 3,
    'third born': 4,
    'fourth born': 5,
    'fiifth born': 6,
    'sixth born or later, not the youngest': 7,
    'sixth born of later, the youngest': 8,

    'Both no college': 1,
    '1 parent no college': 2,
    'both parent college': 3,

    'Latino': 1,
    'Black': 2,
    'Asian-American': 3,
    'White': 4,
    'Other': 5,
    'multiracial': 6,

    'Catholic': 1,
    'Protestant': 2,
    'Other Religion': 3,
    'No Religion': 4,

    'not anxious': 0,
    'anxious': 1,

    'Not depressed': 0,
    'Depressed': 1,
    
    'low': 0,
    'moderate': 1,
    'severe': 2,

}

for col in survey:
  for i, value in survey[col].iteritems():
    if value in scores :
      survey[col][i] = scores[value]

survey.to_csv('/content/drive/MyDrive/senior_project/survey_data_preprocessed.csv', index=False)

In [29]:
survey = pd.read_csv('/content/drive/MyDrive/senior_project/survey_data_preprocessed.csv', dtype=object)
cols = survey.columns

personality_inventory = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']
big_five = ['BigFive']
education_prefix = ['hs', 'hsgrade', 'apcourses', 'apexams', 'degreeintent', 'major', 'ndfirst']
exercise_prefix = ['exercise', 'clubsports', 'varsitysports', 'swimming']
health_prefix = ['selfesteem', 'SRQE_Ext_', 'SRQE_Introj_', 'SRQE_Ident', 'SelfEff_exercise_scale_', 'SelfEff_diet_scale_', 'selfreg_scale_']
mental_prefix = ['STAITraitTotal_', 'CESDOverall_', 'BAIsum_', 'STAITraitGroup_', 'CESDGroup_', 'BAIgroup_', 'majorevent_']
origin_prefix = ['momdec_', 'momusa_', 'daddec_', 'dadusa_', 'parentstatus_', 'dadage_', 'momage_', 'momusa_', 'numsib_', 'birthorder_', 'parenteduc_', 'dadage_', 'momrace_', 'dadrace_', 'daddec_', 'momrelig_', 'dadrelig_', 'yourelig_']
personal_prefix = ['selsa_rom_', 'selsa_fam_', 'selsa_soc_']
gender_prefix = ['gender_']
bad_habits_prefix = ['usetobacco_', 'usebeer_', 'usewine_', 'usedrugs_', 'usedrugs_prescr_', 'usecaffine_']

stress_target_value = ['Stress_']
sleep_target_value = ['PSQIGlobal_']

column_names = personality_inventory + big_five + education_prefix + health_prefix + mental_prefix + origin_prefix + personal_prefix + gender_prefix + bad_habits_prefix



stress_waves = [4,5,6,8]
sleep_waves = [1,2,3,4,6,8]

added_cols = ['egoid']
for wave in sleep_waves:
  for col in cols:
    if col.endswith(f'_{wave}'):
      for column_name in column_names:
        if col.startswith(column_name):
          added_cols.append(col)  

for wave in sleep_waves:
  if wave != 6 and wave != 8:
    added_cols.append(sleep_target_value[0] + str(wave))

survey[added_cols].to_csv('/content/drive/MyDrive/senior_project/survey_partial_final/sleep_waves_survey_data.csv', index=False)


added_cols = ['egoid']
for wave in stress_waves:
  for col in cols:
    if col.endswith(f'_{wave}'):
      for column_name in column_names:
        if col.startswith(column_name):
          added_cols.append(col)

for wave in stress_waves:
  added_cols.append(stress_target_value[0] + str(wave))  

survey[added_cols].to_csv('/content/drive/MyDrive/senior_project/survey_partial_final/stress_waves_survey_data.csv', index=False)



### Needed columns for all waves are extracted below 

added_cols = ['egoid']
for col in cols:
  for column_name in column_names:
    if col.startswith(column_name):
      added_cols.append(col)

survey[added_cols].to_csv('/content/drive/MyDrive/senior_project/survey_partial_final/all_waves_survey_data.csv', index=False)





In [30]:
### Joining wearable data and survey data

stress_survey_data = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/stress_waves_survey_data.csv', dtype=object)
sleep_survey_data = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/sleep_waves_survey_data.csv', dtype=object)
all_survey_data = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/all_waves_survey_data.csv', dtype=object)

wave_dates = {
    1: '8/3/2015',
    2: '1/31/2016',
    3: '6/20/2016',
    4: '11/1/2016', 
    5: '4/10/2017',
    6: '11/9/2017', 
    7: '4/26/2018', 
    8: '4/12/2018',
    9: '12/31/2019'
}

stress_waves = [4,5,6,8]
sleep_waves = [1,2,3,4,6,8]


df_stress_waves = pd.DataFrame()
cols = stress_survey_data.columns

for wave in stress_waves:
  start_date = wave_dates[wave]
  end_date = wave_dates[wave+1]
  temp_df = merged_df[pd.to_datetime(merged_df['datadate']).between(start_date, end_date)]
  
  added_cols = ['egoid']
  for col in cols:
    if col.endswith(f'_{wave}'):
      added_cols.append(col) 
  temp_df = temp_df.merge(stress_survey_data[added_cols], on=['egoid'], validate='many_to_one')


  added_cols.remove('egoid')
  new_names = {}
  for elm in added_cols:
    new_names[elm] = elm[:-2]

  temp_df.rename(columns = new_names, inplace=True)
  
  df_stress_waves = df_stress_waves.append(temp_df, ignore_index=True)
  
df_stress_waves['Stress'] = df_stress_waves.pop('Stress')



df_sleep_waves = pd.DataFrame()
cols = sleep_survey_data.columns

for wave in sleep_waves:
  start_date = wave_dates[wave]
  end_date = wave_dates[wave+1]
  temp_df = merged_df[pd.to_datetime(merged_df['datadate']).between(start_date, end_date)]
  
  added_cols = ['egoid']
  for col in cols:
    if col.endswith(f'_{wave}'):
      added_cols.append(col) 
  temp_df = temp_df.merge(sleep_survey_data[added_cols], on=['egoid'], validate='many_to_one')


  added_cols.remove('egoid')
  new_names = {}
  for elm in added_cols:
    new_names[elm] = elm[:-2]

  temp_df.rename(columns = new_names, inplace=True)
  
  df_sleep_waves = df_sleep_waves.append(temp_df, ignore_index=True)

df_sleep_waves['PSQIGlobal'] = df_sleep_waves.pop('PSQIGlobal')


df_sleep_waves.to_csv('/content/drive/MyDrive/senior_project/sleep_waves_survey_data.csv', index=False)
df_stress_waves.to_csv('/content/drive/MyDrive/senior_project/stress_waves_survey_data.csv', index=False)



### The code below is for the other sleep dataframe: in this dataframe, the target value is sleep data from 
### fitbit devices for all the terms. Also the PSQIGlobal value is deleted in this dataframe.


df_all_waves = pd.DataFrame()
cols = all_survey_data.columns


for wave in range(1,9):
  start_date = wave_dates[wave]
  end_date = wave_dates[wave+1]
  temp_df = merged_df[pd.to_datetime(merged_df['datadate']).between(start_date, end_date)]
  
  added_cols = ['egoid']
  for col in cols:
    if col.endswith(f'_{wave}'):
      added_cols.append(col) 
  temp_df = temp_df.merge(all_survey_data[added_cols], on=['egoid'], validate='many_to_one')


  added_cols.remove('egoid')
  new_names = {}
  for elm in added_cols:
    new_names[elm] = elm[:-2]

  temp_df.rename(columns = new_names, inplace=True)
  
  df_all_waves = df_all_waves.append(temp_df, ignore_index=True)

fitbit_sleep_parameters = ['datadate','timetobed','timeoutofbed','bedtimedur','minstofallasleep','minsafterwakeup','minsasleep','minsawake','Efficiency']

for parameter in fitbit_sleep_parameters:
  df_all_waves[parameter] = df_all_waves.pop(parameter)


df_all_waves.to_csv('/content/drive/MyDrive/senior_project/all_waves_survey_data.csv', index=False)



In [26]:
# stress_survey_data = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/stress_waves_survey_data.csv', dtype=object)
survey = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/sleep_waves_survey_data.csv', dtype=object)
# survey = pd.read_csv('/content/drive/MyDrive/senior_project/survey_partial_final/all_waves_survey_data.csv', dtype=object)

### For column percentages

# cols = survey.columns

# column_nan_counts = {}

# for col in cols:
#   total_nan = survey[col].isna().sum()
#   column_nan_counts[col] = total_nan / len(survey[col])

# column_nan_counts = dict(sorted(column_nan_counts.items(), reverse=True, key=lambda item: item[1]))  

# for elm in column_nan_counts:
#   print(elm, column_nan_counts[elm])



### For student percentages


momdec_1 0.997229916897507
dadrace_Haw_1 0.997229916897507
dadrace_AmInd_1 0.9958448753462604
momrace_Haw_1 0.9930747922437673
momrace_AmInd_1 0.9916897506925207
daddec_1 0.9889196675900277
daddec_1.1 0.9889196675900277
momrace_Black_1 0.9445983379501385
dadrace_Black_1 0.9335180055401662
major2rc_8 0.9196675900277008
dadrace_Latino_1 0.8961218836565097
dadrace_Asian_1 0.8905817174515236
momrace_Latino_1 0.8878116343490304
momrace_Asian_1 0.8822714681440443
major2rc_6 0.871191135734072
major2rc_4 0.8102493074792244
dadusa_1 0.7659279778393352
major2rc_3 0.7617728531855956
momusa_1 0.7465373961218836
momusa_1.1 0.7465373961218836
CESDOverall_8 0.7451523545706371
CESDGroup_8 0.7451523545706371
STAITraitTotal_8 0.7423822714681441
STAITraitGroup_8 0.7423822714681441
BigFive_29_8 0.7382271468144044
BigFive_12_8 0.7368421052631579
BigFive_13_8 0.7368421052631579
BigFive_14_8 0.7368421052631579
BigFive_15_8 0.7368421052631579
BigFive_16_8 0.7368421052631579
BigFive_17_8 0.7368421052631579
Big